In [1]:
import spotipy
import json
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from credentials import *
import pprint
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import clear_output
import random
import time

In [2]:
hot_100_songs = pd.read_csv('hot_100_songs.csv')
new_column = 'hot'
hot_100_songs['hot_or_not'] = new_column
hot_100_songs.head()

,Title,Artist,hot_or_not
0,Last Night,Morgan Wallen,hot
1,Fast Car,Luke Combs,hot
2,Vampire,Olivia Rodrigo,hot
3,Calm Down,Rema & Selena Gomez,hot
4,I Can See You (Taylor's Version) (From The Vault),Taylor Swift,hot


In [3]:
not_hot_songs = pd.read_csv('not_hot_songs.csv')
new_column = 'not_hot'
not_hot_songs['hot_or_not'] = new_column
not_hot_songs.head()

,Title,Artist,hot_or_not
0,I Didn't Mean To,Casual,not_hot
1,Soul Deep,The Box Tops,not_hot
2,Amor De Cabaret,Sonora Santanera,not_hot
3,Something Girls,Adam Ant,not_hot
4,Face the Ashes,Gob,not_hot


In [106]:
all_songs = pd.concat([hot_100_songs, not_hot_songs])
all_songs

,Title,Artist,hot_or_not
0,Last Night,Morgan Wallen,hot
1,Fast Car,Luke Combs,hot
2,Vampire,Olivia Rodrigo,hot
3,Calm Down,Rema & Selena Gomez,hot
4,I Can See You (Taylor's Version) (From The Vault),Taylor Swift,hot
...,...,...,...
9995,The Hanged Man,Moonspell,not_hot
9996,The Wonderful World Of The Young,Danny Williams,not_hot
9997,Sentimental Man,Winston Reedy,not_hot
9998,Zydeco In D-Minor,"Myrick ""Freeze"" Guillory",not_hot


In [26]:
def search_song_api(Title: str, Artist: str):
    
    """
    Gets a tuple (Title, Artist) and retrieves it will retrieve a list of possible songs, from the Spotify API for the user to choose from.
    Then will give us its ID. 
    If the Song isn't in the API, it will remove the song from the databases, printing an empty string as ID.
    
    Args:
    
    Title, Artist = Tuple
    
    Returns:
    
    A list of dictionaries to the user to choose from, and to the program an ID to use later
    """
    
    
    # searching info from API
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

    query = ("track:"+Title if Title != "" else "")+" "+("artist:"+Artist if Artist != "" else "")
    
    if len(query) > 0:
        try:
            raw_results = sp.search(q=query,type="track,artist",limit=5)
            raw_results = raw_results['tracks']['items'] 
        except Exception as e:
            print("Error:", e)
    else:
        raw_results = []
        
    clean_results = []  
    
    for index, item in enumerate(raw_results):
        clean_results.append({'track_id': item['id'],
                        'href': item['href'],
                        'track_link': item['external_urls']['spotify'],
                        'track_name': item['name'],
                        'artist': item['album']['artists'][0]['name'],
                        'album_name': item['album']['name'],
                        'album_release_year': item['album']['release_date'],
                        'track_duration': item['duration_ms'],
                        'album_image': item['album']['images'][1]['url']})
        
    return clean_results

In [27]:
search_song_api('hello', 'adele') #does it work?

[{'track_id': '62PaSfnXSMyLshYJrlTuL3',
  'href': 'https://api.spotify.com/v1/tracks/62PaSfnXSMyLshYJrlTuL3',
  'track_link': 'https://open.spotify.com/track/62PaSfnXSMyLshYJrlTuL3',
  'track_name': 'Hello',
  'artist': 'Adele',
  'album_name': '25',
  'album_release_year': '2015-11-20',
  'track_duration': 295502,
  'album_image': 'https://i.scdn.co/image/ab67616d00001e0247ce408fb4926d69da6713c2'},
 {'track_id': '3AuzZHPlohKLpildLyORSM',
  'href': 'https://api.spotify.com/v1/tracks/3AuzZHPlohKLpildLyORSM',
  'track_link': 'https://open.spotify.com/track/3AuzZHPlohKLpildLyORSM',
  'track_name': 'Hello',
  'artist': 'Adele',
  'album_name': 'Hello',
  'album_release_year': '2015-10-23',
  'track_duration': 295502,
  'album_image': 'https://i.scdn.co/image/ab67616d00001e0223d302c1290685258a400ad0'},
 {'track_id': '2YGaQsuLHn3Qd0ADHNb9gh',
  'href': 'https://api.spotify.com/v1/tracks/2YGaQsuLHn3Qd0ADHNb9gh',
  'track_link': 'https://open.spotify.com/track/2YGaQsuLHn3Qd0ADHNb9gh',
  'track

In [28]:
def get_song_data(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Given a DataFrame with three columns: title and artist and hot_or_not,
    this function returns a new df with track data 
    using Spotipy, Spotify's API.
    
    Input:
    df: pandas DataFrame with songs
    
    Output:
    DataFrame with the track data
    
    '''
    
    # Define loop pauses to avoid API server blockage by overflow
    sleep_time = 15 # seconds of sleep time
    loop_count = 100 # tracks per sleep loop
    
    # Define an empty array to append results
    sp_songs = []
    
    loop_counter = 0 # counter to track loop
    counter = 0 # counter to track progress
    fail_counter = 0 # counter to track failed matches
                 
    for index, row in df.iterrows():
        
        counter += 1 # Add one to the global song counter
        
        # Clear last print and print next progress
        clear_output(wait=True)
        print('Downloading song Data...', round(((index + 1) / df.shape[0])*100), '%')
        
        # Get the current row song title and artist. Fix to maximum 30 chars to avoid API error.
        song_name = str(row['Title'])[:30]
        artist_name = str(row['Artist'])[:30]
        
        try:
            sp_data = search_song_api(song_name, artist_name) # Call search_song function to get data
            if len(sp_data) > 0:
                sp_songs.append(sp_data[0]) # If there is data, append to global variable
            else:
                fail_counter += 1 # If there is no data, add 1 to fail_counter
        except Exception as e:
            # Log error and add counter
            print("Error:", e)
            fail_counter += 1
        
        # Sleep to avoid API overflow...
        loop_counter += 1
        if loop_counter >= loop_count:
            loop_counter = 0
            # Generate a random sleep timer based on sleep_time
            random_sleep_timer = random.randint(int(sleep_time*0.5), int(sleep_time*1.5))
            print('Sleeping for', random_sleep_timer, 's...') # Print sleep time
            time.sleep(random_sleep_timer) # Sleep -> pause download to simulate human behavior

    # Print download summary
    clear_output(wait=True)
    print("Song data download complete.", "Success rate:", round(((counter-fail_counter)/counter)*100), "%")
    print("Succesful downloads:",counter-fail_counter,"Failed downloads:",fail_counter)
    
    # Convert list of dictionaries to DataFrame
    sp_songs = pd.DataFrame(sp_songs)
    
    # Drop result_index column: used for single songs searches in search_song() function
    #sp_songs.drop(inplace=True, columns='result_index')
    
    return sp_songs

In [29]:
sp_songs = get_song_data(all_songs)

Song data download complete. Success rate: 71 %
Succesful downloads: 7125 Failed downloads: 2975


In [30]:
sp_songs.head()

,track_id,href,track_link,track_name,artist,album_name,album_release_year,track_duration,album_image
0,59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://open.spotify.com/track/59uQI0PADDKeE6U...,Last Night,Morgan Wallen,3 Songs At A Time Sampler,2023-01-31,163854,https://i.scdn.co/image/ab67616d00001e02fc1df8...
1,1Lo0QY9cvc8sUB2vnIOxDT,https://api.spotify.com/v1/tracks/1Lo0QY9cvc8s...,https://open.spotify.com/track/1Lo0QY9cvc8sUB2...,Fast Car,Luke Combs,Gettin' Old,2023-03-24,265493,https://i.scdn.co/image/ab67616d00001e02ca650d...
2,3k79jB4aGmMDUQzEwa46Rz,https://api.spotify.com/v1/tracks/3k79jB4aGmMD...,https://open.spotify.com/track/3k79jB4aGmMDUQz...,vampire,Olivia Rodrigo,vampire,2023-06-30,219724,https://i.scdn.co/image/ab67616d00001e021e5e75...
3,1s7oOCT8vauUh01PbJD6ps,https://api.spotify.com/v1/tracks/1s7oOCT8vauU...,https://open.spotify.com/track/1s7oOCT8vauUh01...,Calm Down (with Selena Gomez),Rema,Rave & Roses Ultra,2023-04-27,239317,https://i.scdn.co/image/ab67616d00001e02963265...
4,5kHMfzgLZP95O9NBy0ku4v,https://api.spotify.com/v1/tracks/5kHMfzgLZP95...,https://open.spotify.com/track/5kHMfzgLZP95O9N...,I Can See You (Taylor’s Version) (From The Vault),Taylor Swift,Speak Now (Taylor's Version),2023-07-07,273186,https://i.scdn.co/image/ab67616d00001e020b04da...


In [32]:
# Create list of track_ids from all song data
track_ids = list(sp_songs['track_id'])

In [35]:
# Create a function to search a given single song in the Spotify API: search_song(title, artist).

def get_audio_features(track_ids: list) -> pd.DataFrame:
    '''
    Given a list of track IDs, this function returns 
    track audio features such as danceability, energy or tempo
    using Spotipy, Spotify's API.
    
    Input:
    track_ids: list of Spotipy track_ids
    
    Output:
    DataFrame with the track_ids and audio features
    
    '''
    
    # Define loop pauses to avoid API server blockage by overflow
    sleep_time = 20 # seconds of sleep time
    loop_count = 100 # tracks per sleep loop
    
    # Initialize SpotiPy with user credentias
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))

    # Define result variable
    clean_results = []
    
    loop_counter = 0 # counter to track loop
    counter = 0 # counter to track progress
    fail_counter = 0 # counter to track failed matches
    
    # Loop through track_ids to get relevant audio features
    for index, track_id in enumerate(track_ids):
        
        counter += 1 # Add 1 to global counter
        clear_output(wait=True) # Clear print output to update progress
        print('Downloading audio features...', round(((index + 1) / len(track_ids))*100), '%') # Print progress
        
        try:
            # Get audio features from Spotipy
            api_result = sp.audio_features(track_id)[0]
            
            # Add features to track variable
            track = {'track_id': track_id,
                     'acousticness': api_result['acousticness'],
                     'danceability': api_result['danceability'],
                     'energy': api_result['energy'],
                     'instrumentalness': api_result['instrumentalness'],
                     'key': api_result['key'],
                     'liveness': api_result['liveness'],
                     'loudness': api_result['loudness'],
                     'mode': api_result['mode'],
                     'speechiness': api_result['speechiness'],
                     'tempo': api_result['tempo'],
                     'time_signature': api_result['time_signature'],
                     'valence': api_result['valence'],
            }
            
            # Append track results to results list
            clean_results.append(track)
            
        except Exception as e:
            # If there is an error:
            print("Error:", e)
            fail_counter += 1
        
        # Sleep to avoid API overflow...
        loop_counter += 1
        if loop_counter >= loop_count:
            loop_counter = 0
            # Generate a random sleep timer based on sleep_time
            random_sleep_timer = random.randint(int(sleep_time*0.5), int(sleep_time*1.5))
            print('Sleeping for', random_sleep_timer, 's...')
            time.sleep(random_sleep_timer) # Sleep
            
    
    clear_output(wait=True)
    print("Audio features download complete.", "Success rate:", round(((counter-fail_counter)/counter)*100), "%")
    print("Succesful downloads:",counter-fail_counter,"Failed downloads:",fail_counter)
    
    # Convert list of dictionaries to DataFrame
    clean_results = pd.DataFrame(clean_results)

    return clean_results

In [36]:
# Download audio features data
audio_features = get_audio_features(track_ids)

Audio features download complete. Success rate: 100 %
Succesful downloads: 7122 Failed downloads: 3


In [37]:
# Create function to concatenate dataframes

def add_audio_features(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    '''
    This function concatenates the audio features columns to the base 
    song DataFrame columns and removes any duplicate columns of the 
    concatenates DataFrame.
    
    Input:
    df: base songs DataFrame
    audio_features
    
    '''
    
    # Merge datasets based on track_id column
    merged_df = pd.merge(df1, df2, on='track_id', how='inner')
    
    # Remove duplicate columns
    merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
    
    return merged_df

In [136]:
full_song_data = add_audio_features(sp_songs, audio_features)
full_song_data

,track_id,href,track_link,track_name,artist,album_name,album_release_year,track_duration,album_image,acousticness,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://open.spotify.com/track/59uQI0PADDKeE6U...,Last Night,Morgan Wallen,3 Songs At A Time Sampler,2023-01-31,163854,https://i.scdn.co/image/ab67616d00001e02fc1df8...,0.459000,...,0.675,0.000000,6,0.1510,-5.382,1,0.0357,203.853,4,0.5180
1,1Lo0QY9cvc8sUB2vnIOxDT,https://api.spotify.com/v1/tracks/1Lo0QY9cvc8s...,https://open.spotify.com/track/1Lo0QY9cvc8sUB2...,Fast Car,Luke Combs,Gettin' Old,2023-03-24,265493,https://i.scdn.co/image/ab67616d00001e02ca650d...,0.186000,...,0.603,0.000000,8,0.1150,-5.520,1,0.0262,97.994,4,0.6700
2,3k79jB4aGmMDUQzEwa46Rz,https://api.spotify.com/v1/tracks/3k79jB4aGmMD...,https://open.spotify.com/track/3k79jB4aGmMDUQz...,vampire,Olivia Rodrigo,vampire,2023-06-30,219724,https://i.scdn.co/image/ab67616d00001e021e5e75...,0.169000,...,0.532,0.000000,5,0.3110,-5.745,1,0.0560,137.827,4,0.3220
3,1s7oOCT8vauUh01PbJD6ps,https://api.spotify.com/v1/tracks/1s7oOCT8vauU...,https://open.spotify.com/track/1s7oOCT8vauUh01...,Calm Down (with Selena Gomez),Rema,Rave & Roses Ultra,2023-04-27,239317,https://i.scdn.co/image/ab67616d00001e02963265...,0.429000,...,0.802,0.001280,11,0.1710,-5.196,1,0.0371,107.008,4,0.8110
4,5kHMfzgLZP95O9NBy0ku4v,https://api.spotify.com/v1/tracks/5kHMfzgLZP95...,https://open.spotify.com/track/5kHMfzgLZP95O9N...,I Can See You (Taylor’s Version) (From The Vault),Taylor Swift,Speak Now (Taylor's Version),2023-07-07,273186,https://i.scdn.co/image/ab67616d00001e020b04da...,0.058600,...,0.764,0.000000,6,0.0608,-4.893,1,0.0337,123.044,4,0.8190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7257,3e0tyTV5FiV1bcYeRjdDz2,https://api.spotify.com/v1/tracks/3e0tyTV5FiV1...,https://open.spotify.com/track/3e0tyTV5FiV1bcY...,The Hanged Man,Moonspell,Sin / Pecado,1998,386333,https://i.scdn.co/image/ab67616d00001e02c69de1...,0.064300,...,0.636,0.653000,4,0.0795,-8.798,0,0.0325,140.105,4,0.0979
7258,62wqW6Q9eTozrruWPt9Z9i,https://api.spotify.com/v1/tracks/62wqW6Q9eToz...,https://open.spotify.com/track/62wqW6Q9eTozrru...,The Wonderful World of the Young,Danny Williams,Collection,1997,169333,https://i.scdn.co/image/ab67616d00001e02723684...,0.944000,...,0.251,0.000000,8,0.0997,-12.959,1,0.0371,179.310,1,0.3100
7259,7n0ShZeoewT4ZOJCmwHdnQ,https://api.spotify.com/v1/tracks/7n0ShZeoewT4...,https://open.spotify.com/track/7n0ShZeoewT4ZOJ...,Sentimental Man,Winston Reedy,Reality,2002-01-01,196200,https://i.scdn.co/image/ab67616d00001e02dbf5ac...,0.023600,...,0.395,0.000084,11,0.0654,-11.838,1,0.0653,118.089,4,0.9220
7260,5DqBIhTIAOUQRsmqe5f0oA,https://api.spotify.com/v1/tracks/5DqBIhTIAOUQ...,https://open.spotify.com/track/5DqBIhTIAOUQRsm...,Zydeco In D-Minor,"Myrick ""Freeze"" Guillory",Nouveau Zydeco,1991,300773,https://i.scdn.co/image/ab67616d00001e026476f9...,0.084400,...,0.771,0.000762,0,0.9870,-12.679,1,0.2240,139.026,4,0.6850


In [137]:
all_songs.rename(columns={'Title': 'track_name'}, inplace=True)
all_songs.rename(columns={'Artist':'artist'}, inplace=True)
full_song_data = pd.merge(full_song_data, all_songs, on=('track_name'), how='inner')
full_song_data

,track_id,href,track_link,track_name,artist_x,album_name,album_release_year,track_duration,album_image,acousticness,...,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist_y,hot_or_not
0,59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://open.spotify.com/track/59uQI0PADDKeE6U...,Last Night,Morgan Wallen,3 Songs At A Time Sampler,2023-01-31,163854,https://i.scdn.co/image/ab67616d00001e02fc1df8...,0.459000,...,6,0.1510,-5.382,1,0.0357,203.853,4,0.5180,Morgan Wallen,hot
1,59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://open.spotify.com/track/59uQI0PADDKeE6U...,Last Night,Morgan Wallen,3 Songs At A Time Sampler,2023-01-31,163854,https://i.scdn.co/image/ab67616d00001e02fc1df8...,0.459000,...,6,0.1510,-5.382,1,0.0357,203.853,4,0.5180,Northern State,not_hot
2,1Lo0QY9cvc8sUB2vnIOxDT,https://api.spotify.com/v1/tracks/1Lo0QY9cvc8s...,https://open.spotify.com/track/1Lo0QY9cvc8sUB2...,Fast Car,Luke Combs,Gettin' Old,2023-03-24,265493,https://i.scdn.co/image/ab67616d00001e02ca650d...,0.186000,...,8,0.1150,-5.520,1,0.0262,97.994,4,0.6700,Luke Combs,hot
3,0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/tracks/0yLdNVWF3Sre...,https://open.spotify.com/track/0yLdNVWF3Srea0u...,Flowers,Miley Cyrus,Flowers,2023-01-13,200454,https://i.scdn.co/image/ab67616d00001e02f42954...,0.063200,...,0,0.0322,-4.325,1,0.0668,117.999,4,0.6460,Miley Cyrus,hot
4,1BxfuPKGuaTgP7aM0Bbdwr,https://api.spotify.com/v1/tracks/1BxfuPKGuaTg...,https://open.spotify.com/track/1BxfuPKGuaTgP7a...,Cruel Summer,Taylor Swift,Lover,2019-08-23,178426,https://i.scdn.co/image/ab67616d00001e02e787cf...,0.117000,...,9,0.1050,-5.707,1,0.1570,169.994,4,0.5640,Taylor Swift,hot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5557,1Lm0nXUfl9WlMTPvUQPLVk,https://api.spotify.com/v1/tracks/1Lm0nXUfl9Wl...,https://open.spotify.com/track/1Lm0nXUfl9WlMTP...,God Don't Never Change,Blind Willie Johnson,The Complete Blind Willie Johnson,1993,176600,https://i.scdn.co/image/ab67616d00001e02bdee69...,0.957000,...,0,0.3070,-15.811,1,0.0626,167.944,4,0.5780,Blind Willie Johnson,not_hot
5558,3e0tyTV5FiV1bcYeRjdDz2,https://api.spotify.com/v1/tracks/3e0tyTV5FiV1...,https://open.spotify.com/track/3e0tyTV5FiV1bcY...,The Hanged Man,Moonspell,Sin / Pecado,1998,386333,https://i.scdn.co/image/ab67616d00001e02c69de1...,0.064300,...,4,0.0795,-8.798,0,0.0325,140.105,4,0.0979,Moonspell,not_hot
5559,7n0ShZeoewT4ZOJCmwHdnQ,https://api.spotify.com/v1/tracks/7n0ShZeoewT4...,https://open.spotify.com/track/7n0ShZeoewT4ZOJ...,Sentimental Man,Winston Reedy,Reality,2002-01-01,196200,https://i.scdn.co/image/ab67616d00001e02dbf5ac...,0.023600,...,11,0.0654,-11.838,1,0.0653,118.089,4,0.9220,Winston Reedy,not_hot
5560,5DqBIhTIAOUQRsmqe5f0oA,https://api.spotify.com/v1/tracks/5DqBIhTIAOUQ...,https://open.spotify.com/track/5DqBIhTIAOUQRsm...,Zydeco In D-Minor,"Myrick ""Freeze"" Guillory",Nouveau Zydeco,1991,300773,https://i.scdn.co/image/ab67616d00001e026476f9...,0.084400,...,0,0.9870,-12.679,1,0.2240,139.026,4,0.6850,"Myrick ""Freeze"" Guillory",not_hot


In [138]:
full_song_data = full_song_data.drop('artist_y', axis=1)

In [139]:
full_song_data

,track_id,href,track_link,track_name,artist_x,album_name,album_release_year,track_duration,album_image,acousticness,...,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,hot_or_not
0,59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://open.spotify.com/track/59uQI0PADDKeE6U...,Last Night,Morgan Wallen,3 Songs At A Time Sampler,2023-01-31,163854,https://i.scdn.co/image/ab67616d00001e02fc1df8...,0.459000,...,0.000000,6,0.1510,-5.382,1,0.0357,203.853,4,0.5180,hot
1,59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://open.spotify.com/track/59uQI0PADDKeE6U...,Last Night,Morgan Wallen,3 Songs At A Time Sampler,2023-01-31,163854,https://i.scdn.co/image/ab67616d00001e02fc1df8...,0.459000,...,0.000000,6,0.1510,-5.382,1,0.0357,203.853,4,0.5180,not_hot
2,1Lo0QY9cvc8sUB2vnIOxDT,https://api.spotify.com/v1/tracks/1Lo0QY9cvc8s...,https://open.spotify.com/track/1Lo0QY9cvc8sUB2...,Fast Car,Luke Combs,Gettin' Old,2023-03-24,265493,https://i.scdn.co/image/ab67616d00001e02ca650d...,0.186000,...,0.000000,8,0.1150,-5.520,1,0.0262,97.994,4,0.6700,hot
3,0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/tracks/0yLdNVWF3Sre...,https://open.spotify.com/track/0yLdNVWF3Srea0u...,Flowers,Miley Cyrus,Flowers,2023-01-13,200454,https://i.scdn.co/image/ab67616d00001e02f42954...,0.063200,...,0.000005,0,0.0322,-4.325,1,0.0668,117.999,4,0.6460,hot
4,1BxfuPKGuaTgP7aM0Bbdwr,https://api.spotify.com/v1/tracks/1BxfuPKGuaTg...,https://open.spotify.com/track/1BxfuPKGuaTgP7a...,Cruel Summer,Taylor Swift,Lover,2019-08-23,178426,https://i.scdn.co/image/ab67616d00001e02e787cf...,0.117000,...,0.000021,9,0.1050,-5.707,1,0.1570,169.994,4,0.5640,hot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5557,1Lm0nXUfl9WlMTPvUQPLVk,https://api.spotify.com/v1/tracks/1Lm0nXUfl9Wl...,https://open.spotify.com/track/1Lm0nXUfl9WlMTP...,God Don't Never Change,Blind Willie Johnson,The Complete Blind Willie Johnson,1993,176600,https://i.scdn.co/image/ab67616d00001e02bdee69...,0.957000,...,0.000003,0,0.3070,-15.811,1,0.0626,167.944,4,0.5780,not_hot
5558,3e0tyTV5FiV1bcYeRjdDz2,https://api.spotify.com/v1/tracks/3e0tyTV5FiV1...,https://open.spotify.com/track/3e0tyTV5FiV1bcY...,The Hanged Man,Moonspell,Sin / Pecado,1998,386333,https://i.scdn.co/image/ab67616d00001e02c69de1...,0.064300,...,0.653000,4,0.0795,-8.798,0,0.0325,140.105,4,0.0979,not_hot
5559,7n0ShZeoewT4ZOJCmwHdnQ,https://api.spotify.com/v1/tracks/7n0ShZeoewT4...,https://open.spotify.com/track/7n0ShZeoewT4ZOJ...,Sentimental Man,Winston Reedy,Reality,2002-01-01,196200,https://i.scdn.co/image/ab67616d00001e02dbf5ac...,0.023600,...,0.000084,11,0.0654,-11.838,1,0.0653,118.089,4,0.9220,not_hot
5560,5DqBIhTIAOUQRsmqe5f0oA,https://api.spotify.com/v1/tracks/5DqBIhTIAOUQ...,https://open.spotify.com/track/5DqBIhTIAOUQRsm...,Zydeco In D-Minor,"Myrick ""Freeze"" Guillory",Nouveau Zydeco,1991,300773,https://i.scdn.co/image/ab67616d00001e026476f9...,0.084400,...,0.000762,0,0.9870,-12.679,1,0.2240,139.026,4,0.6850,not_hot


In [140]:
full_song_data.rename(columns={'artist_x': 'artist'}, inplace=True)
full_song_data

,track_id,href,track_link,track_name,artist,album_name,album_release_year,track_duration,album_image,acousticness,...,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,hot_or_not
0,59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://open.spotify.com/track/59uQI0PADDKeE6U...,Last Night,Morgan Wallen,3 Songs At A Time Sampler,2023-01-31,163854,https://i.scdn.co/image/ab67616d00001e02fc1df8...,0.459000,...,0.000000,6,0.1510,-5.382,1,0.0357,203.853,4,0.5180,hot
1,59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://open.spotify.com/track/59uQI0PADDKeE6U...,Last Night,Morgan Wallen,3 Songs At A Time Sampler,2023-01-31,163854,https://i.scdn.co/image/ab67616d00001e02fc1df8...,0.459000,...,0.000000,6,0.1510,-5.382,1,0.0357,203.853,4,0.5180,not_hot
2,1Lo0QY9cvc8sUB2vnIOxDT,https://api.spotify.com/v1/tracks/1Lo0QY9cvc8s...,https://open.spotify.com/track/1Lo0QY9cvc8sUB2...,Fast Car,Luke Combs,Gettin' Old,2023-03-24,265493,https://i.scdn.co/image/ab67616d00001e02ca650d...,0.186000,...,0.000000,8,0.1150,-5.520,1,0.0262,97.994,4,0.6700,hot
3,0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/tracks/0yLdNVWF3Sre...,https://open.spotify.com/track/0yLdNVWF3Srea0u...,Flowers,Miley Cyrus,Flowers,2023-01-13,200454,https://i.scdn.co/image/ab67616d00001e02f42954...,0.063200,...,0.000005,0,0.0322,-4.325,1,0.0668,117.999,4,0.6460,hot
4,1BxfuPKGuaTgP7aM0Bbdwr,https://api.spotify.com/v1/tracks/1BxfuPKGuaTg...,https://open.spotify.com/track/1BxfuPKGuaTgP7a...,Cruel Summer,Taylor Swift,Lover,2019-08-23,178426,https://i.scdn.co/image/ab67616d00001e02e787cf...,0.117000,...,0.000021,9,0.1050,-5.707,1,0.1570,169.994,4,0.5640,hot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5557,1Lm0nXUfl9WlMTPvUQPLVk,https://api.spotify.com/v1/tracks/1Lm0nXUfl9Wl...,https://open.spotify.com/track/1Lm0nXUfl9WlMTP...,God Don't Never Change,Blind Willie Johnson,The Complete Blind Willie Johnson,1993,176600,https://i.scdn.co/image/ab67616d00001e02bdee69...,0.957000,...,0.000003,0,0.3070,-15.811,1,0.0626,167.944,4,0.5780,not_hot
5558,3e0tyTV5FiV1bcYeRjdDz2,https://api.spotify.com/v1/tracks/3e0tyTV5FiV1...,https://open.spotify.com/track/3e0tyTV5FiV1bcY...,The Hanged Man,Moonspell,Sin / Pecado,1998,386333,https://i.scdn.co/image/ab67616d00001e02c69de1...,0.064300,...,0.653000,4,0.0795,-8.798,0,0.0325,140.105,4,0.0979,not_hot
5559,7n0ShZeoewT4ZOJCmwHdnQ,https://api.spotify.com/v1/tracks/7n0ShZeoewT4...,https://open.spotify.com/track/7n0ShZeoewT4ZOJ...,Sentimental Man,Winston Reedy,Reality,2002-01-01,196200,https://i.scdn.co/image/ab67616d00001e02dbf5ac...,0.023600,...,0.000084,11,0.0654,-11.838,1,0.0653,118.089,4,0.9220,not_hot
5560,5DqBIhTIAOUQRsmqe5f0oA,https://api.spotify.com/v1/tracks/5DqBIhTIAOUQ...,https://open.spotify.com/track/5DqBIhTIAOUQRsm...,Zydeco In D-Minor,"Myrick ""Freeze"" Guillory",Nouveau Zydeco,1991,300773,https://i.scdn.co/image/ab67616d00001e026476f9...,0.084400,...,0.000762,0,0.9870,-12.679,1,0.2240,139.026,4,0.6850,not_hot


In [142]:
full_song_data.drop_duplicates()

,track_id,href,track_link,track_name,artist,album_name,album_release_year,track_duration,album_image,acousticness,...,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,hot_or_not
0,59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://open.spotify.com/track/59uQI0PADDKeE6U...,Last Night,Morgan Wallen,3 Songs At A Time Sampler,2023-01-31,163854,https://i.scdn.co/image/ab67616d00001e02fc1df8...,0.459000,...,0.000000,6,0.1510,-5.382,1,0.0357,203.853,4,0.5180,hot
1,59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://open.spotify.com/track/59uQI0PADDKeE6U...,Last Night,Morgan Wallen,3 Songs At A Time Sampler,2023-01-31,163854,https://i.scdn.co/image/ab67616d00001e02fc1df8...,0.459000,...,0.000000,6,0.1510,-5.382,1,0.0357,203.853,4,0.5180,not_hot
2,1Lo0QY9cvc8sUB2vnIOxDT,https://api.spotify.com/v1/tracks/1Lo0QY9cvc8s...,https://open.spotify.com/track/1Lo0QY9cvc8sUB2...,Fast Car,Luke Combs,Gettin' Old,2023-03-24,265493,https://i.scdn.co/image/ab67616d00001e02ca650d...,0.186000,...,0.000000,8,0.1150,-5.520,1,0.0262,97.994,4,0.6700,hot
3,0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/tracks/0yLdNVWF3Sre...,https://open.spotify.com/track/0yLdNVWF3Srea0u...,Flowers,Miley Cyrus,Flowers,2023-01-13,200454,https://i.scdn.co/image/ab67616d00001e02f42954...,0.063200,...,0.000005,0,0.0322,-4.325,1,0.0668,117.999,4,0.6460,hot
4,1BxfuPKGuaTgP7aM0Bbdwr,https://api.spotify.com/v1/tracks/1BxfuPKGuaTg...,https://open.spotify.com/track/1BxfuPKGuaTgP7a...,Cruel Summer,Taylor Swift,Lover,2019-08-23,178426,https://i.scdn.co/image/ab67616d00001e02e787cf...,0.117000,...,0.000021,9,0.1050,-5.707,1,0.1570,169.994,4,0.5640,hot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5557,1Lm0nXUfl9WlMTPvUQPLVk,https://api.spotify.com/v1/tracks/1Lm0nXUfl9Wl...,https://open.spotify.com/track/1Lm0nXUfl9WlMTP...,God Don't Never Change,Blind Willie Johnson,The Complete Blind Willie Johnson,1993,176600,https://i.scdn.co/image/ab67616d00001e02bdee69...,0.957000,...,0.000003,0,0.3070,-15.811,1,0.0626,167.944,4,0.5780,not_hot
5558,3e0tyTV5FiV1bcYeRjdDz2,https://api.spotify.com/v1/tracks/3e0tyTV5FiV1...,https://open.spotify.com/track/3e0tyTV5FiV1bcY...,The Hanged Man,Moonspell,Sin / Pecado,1998,386333,https://i.scdn.co/image/ab67616d00001e02c69de1...,0.064300,...,0.653000,4,0.0795,-8.798,0,0.0325,140.105,4,0.0979,not_hot
5559,7n0ShZeoewT4ZOJCmwHdnQ,https://api.spotify.com/v1/tracks/7n0ShZeoewT4...,https://open.spotify.com/track/7n0ShZeoewT4ZOJ...,Sentimental Man,Winston Reedy,Reality,2002-01-01,196200,https://i.scdn.co/image/ab67616d00001e02dbf5ac...,0.023600,...,0.000084,11,0.0654,-11.838,1,0.0653,118.089,4,0.9220,not_hot
5560,5DqBIhTIAOUQRsmqe5f0oA,https://api.spotify.com/v1/tracks/5DqBIhTIAOUQ...,https://open.spotify.com/track/5DqBIhTIAOUQRsm...,Zydeco In D-Minor,"Myrick ""Freeze"" Guillory",Nouveau Zydeco,1991,300773,https://i.scdn.co/image/ab67616d00001e026476f9...,0.084400,...,0.000762,0,0.9870,-12.679,1,0.2240,139.026,4,0.6850,not_hot


In [143]:
full_song_data.to_csv('full_song_data.csv', index=False)